# Hyperopt LR SQGL

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from fit_params.fit_functions import *

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter

from custompackage.load_architecture import *
from custompackage.load_data import *
from custompackage.traintestloop import *

import sparselinear as sl
import custompackage.sl_custom as slc

import hyperopt as hp
%matplotlib inline

np.set_printoptions(suppress=True)

import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [3]:
class Hyperoptimizer():
    '''
    - Includes objective function methods
    - Includes run trials
    - Includes "while true" loop
    - Parameters will be: dataset, file location, types of hyperparameters optimized
    - Activation functions: ReLU, LReLU, Sigmoid, SiLU, Swish, SQGL
    - Learning-rate focused
    '''
    def __init__(self, model_type='ktree', selector=0, dataset='mnist', DIR='./results/temp/', threshold=300, Synapse=False,
                 activation='sqgl', leak=0.01, positive=False):
        self.model_type = model_type
        self.selector = selector
        self.dataset = dataset
        self.DIR = DIR
        self.threshold = threshold
        self.Synapse = Synapse
        self.activation = activation
        self.leak = leak
        self.positive = positive
        
        if self.Synapse == True:
            self.syn_key = 'syn'
        else:
            self.syn_key = 'nosyn'
            
        if self.positive == True:
            self.pos_key = 'pos'
        else:
            self.pos_key = 'posneg'
        
        self.space = hp.choice('a',
                                [
                                    {
                                        'lr': hp.uniform('lr', 0.0001, 0.01)
                                    }
                                ])
    
    def objective(self, params):
        
        #Hyperparameter to be optimized
        lr = params['lr']
        
        #SQGL parameters if appropriate

        atten = 0.5
        scale = 1
        
        # Initialize settings
        bs = 256
        weighting = 'paired'
        trials = 10
        epochs = 2000
        trees_set = [1]
        trees = trees_set[0]

        # Load class-dataset list
        classes = np.load('./results/classes.npy', allow_pickle=True)
        classes = [classes[self.selector]]


        # For each dataset enumerated from classes list
        for j, (t1, t2, ds) in enumerate(classes):
            print(t1, t2, ds)
            # Load data loaders
            trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds, permute=False)
            # Initialize input size for model initialization purposes
            input_size = trainloaders[0].dataset.tensors[0][0].shape[0]
            # For each trial
            # Initialize the model
            if self.model_type == 'fcnn':
                if self.Synapse == True:
                    model = synapse_fcnn(Input_size=input_size, Hidden_size=trees*2, Output_size=1,
                                         Activation=self.activation, learn=False, alpha=1, beta=1, gamma=1, 
                                         scale=scale, atten=atten, leak=self.leak).cuda()
                else:
                    model = simple_fcnn(Input_size=input_size, Hidden_size=trees*2, Output_size=1,
                                         Activation=self.activation, learn=False, alpha=1, beta=1, gamma=1, 
                                         scale=scale, atten=atten, leak=self.leak).cuda()
            else:
                if self.Synapse == True:
                    model = ktree_synapse(ds=ds, Repeats=trees, Padded=True, Activation=self.activation, 
                                         learn=False, alpha=1, beta=1, gamma=1, scale=scale, atten=atten,
                                         leak=self.leak, Node_vary=False, positive=self.positive).cuda()
                else:
                    model = ktree_sparse(ds=ds, Repeats=trees, Padded=True, Activation=self.activation, 
                                         learn=False, alpha=1, beta=1, gamma=1, scale=scale, atten=atten,
                                         synapse=False, leak=self.leak, Node_vary=False, positive=self.positive).cuda()
            #Train and test ktree, assigning loss and acc values
            if self.Synapse == True and self.model_type == 'ktree':
                loss_curve, acc_curve, loss, acc, model_t = train_test_ktree_synapse(model, trainloaders[0],
                                                                                validloaders[0], validloaders[0], 
                                                                                epochs = epochs, randorder=False,
                                                                                lr=lr)
            else:
                loss_curve, acc_curve, loss, acc, model_t = train_test_ktree_sparse(model, trainloaders[0],
                                                                                validloaders[0], validloaders[0], 
                                                                                epochs = epochs, randorder=False,
                                                                                lr=lr)
            break
        if loss > 9:
            status = 'fail'
        else:
            status = 'ok'
        return {
            'loss': loss,
            'acc' : acc,
            'status': status,
            }
    
    def run_trials(self):

        trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after iteration
        max_trials = 1  # initial max_trials. put something small to not have to wait

        file_label = '_'.join((self.model_type,self.activation,self.syn_key,self.pos_key,str(self.leak*100),self.dataset))
        
        try:  # try to load an already saved trials object, and increase the max
            trials = pickle.load(open(self.DIR + file_label +'.hyperopt', "rb"))
            print("Found saved Trials! Loading...")
            max_trials = len(trials.trials) + trials_step
            print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
        except:  # create a new trials object and start searching
            trials = Trials()

        best = fmin(fn=self.objective, space=self.space, algo=tpe.suggest, max_evals=max_trials, trials=trials)

        print("Best:", best)

        # save the trials object
        with open(self.DIR + file_label +'.hyperopt', "wb") as f:
            pickle.dump(trials, f)
            
        if max_trials >= self.threshold:
            return(False)
        else:
            return(True)
    
    def loop_to_threshold(self):
        run_status = True
        while run_status:
            run_status = self.run_trials()
        print('Threshold of '+str(self.threshold)+' reached')

## ktree

### nosyn posneg

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=0, 
                                    dataset='mnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=1, 
                                    dataset='fmnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=2, 
                                    dataset='kmnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=3, 
                                    dataset='emnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=4, 
                                    dataset='svhn', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=5, 
                                    dataset='usps', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=6, 
                                    dataset='cifar10', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

### nosyn pos

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=0, 
                                    dataset='mnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=1, 
                                    dataset='fmnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=2, 
                                    dataset='kmnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=3, 
                                    dataset='emnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=4, 
                                    dataset='svhn', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=5, 
                                    dataset='usps', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=6, 
                                    dataset='cifar10', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=False,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

### syn posneg

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=0, 
                                    dataset='mnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=1, 
                                    dataset='fmnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=2, 
                                    dataset='kmnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=3, 
                                    dataset='emnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=4, 
                                    dataset='svhn', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=5, 
                                    dataset='usps', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=6, 
                                    dataset='cifar10', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=False)
hyperoptimizer_syn.loop_to_threshold()

### syn pos

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=0, 
                                    dataset='mnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=1, 
                                    dataset='fmnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=2, 
                                    dataset='kmnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=3, 
                                    dataset='emnist', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=4, 
                                    dataset='svhn', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=5, 
                                    dataset='usps', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

# Initialize hyperoptimizer
hyperoptimizer_syn = Hyperoptimizer(model_type='ktree',
                                    selector=6, 
                                    dataset='cifar10', 
                                    DIR='./results/20210205/', 
                                    threshold=300,
                                    Synapse=True,
                                    activation='sqgl',
                                    leak=0.0,
                                    positive=True)
hyperoptimizer_syn.loop_to_threshold()

## Best Learning Rates

In [7]:
# Get list of classes
classes = np.load('./results/classes.npy', allow_pickle=True)
class_names = classes[:,2]


activations = ['relu', 'lrelu','sigmoid','swish','sqgl']

syn_keys = ['syn', 'nosyn']
pos_keys = ['pos', 'posneg']

# Initialize dictionary
sqgl_dict = {}

# If synapse included, load from correct folder by assigning DIR path
DIR = './results/20201227/'
DIR_sqgl = './results/20210205/'
    
    
# Load best hyperparameters for each dataset
act_dict = {}
for act in activations:

    dataset_dict = {}
    for dataset in class_names:

        syn_dict = {}
        if act == 'sqgl':
            for syn_key in syn_keys:
                pos_dict = {}
                for pos_key in pos_keys:
                    trials = pickle.load(open(DIR_sqgl + 'ktree_'+act+'_'+syn_key+'_'+pos_key+'_0.0_'+dataset+'.hyperopt', "rb"))
                    lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                    pos_dict[pos_key] = {'lr' : lr}
                syn_dict[syn_key] = pos_dict
        else:
            syn_key = 'nosyn'
            
            if act == 'lrelu':
                trials = pickle.load(open(DIR + 'l_ktree_relu_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}
            else:
                trials = pickle.load(open(DIR + 'ktree_'+act+'_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}

        dataset_dict[dataset] = syn_dict

    act_dict[act] = dataset_dict

for act in activations:
    print(act)
    for dataset in class_names:
        
        print(dataset, act_dict[act][dataset])
        
        
with open('./results/hyperparameters/20210207_ktree_lr.hyperopt', 'wb') as handle:
    pickle.dump(act_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

relu
mnist {'nosyn': {'lr': 0.0058}}
fmnist {'nosyn': {'lr': 0.006}}
kmnist {'nosyn': {'lr': 0.001}}
emnist {'nosyn': {'lr': 0.0043}}
svhn {'nosyn': {'lr': 0.0079}}
usps {'nosyn': {'lr': 0.0021}}
cifar10 {'nosyn': {'lr': 0.0024}}
lrelu
mnist {'nosyn': {'lr': 0.0059}}
fmnist {'nosyn': {'lr': 0.0006}}
kmnist {'nosyn': {'lr': 0.0051}}
emnist {'nosyn': {'lr': 0.0018}}
svhn {'nosyn': {'lr': 0.0081}}
usps {'nosyn': {'lr': 0.0056}}
cifar10 {'nosyn': {'lr': 0.004}}
sigmoid
mnist {'nosyn': {'lr': 0.0068}}
fmnist {'nosyn': {'lr': 0.0015}}
kmnist {'nosyn': {'lr': 0.0049}}
emnist {'nosyn': {'lr': 0.0077}}
svhn {'nosyn': {'lr': 0.0012}}
usps {'nosyn': {'lr': 0.0057}}
cifar10 {'nosyn': {'lr': 0.0083}}
swish
mnist {'nosyn': {'lr': 0.0079}}
fmnist {'nosyn': {'lr': 0.0001}}
kmnist {'nosyn': {'lr': 0.0099}}
emnist {'nosyn': {'lr': 0.0018}}
svhn {'nosyn': {'lr': 0.0003}}
usps {'nosyn': {'lr': 0.01}}
cifar10 {'nosyn': {'lr': 0.0002}}
sqgl
mnist {'syn': {'pos': {'lr': 0.0012}, 'posneg': {'lr': 0.0001}}, 'n

## FCNN

### nosyn posneg

In [ ]:
classes = np.load('./results/classes.npy')
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    # Initialize hyperoptimizer
    hyperoptimizer_syn = Hyperoptimizer(model_type='fcnn',
                                        selector=selector, 
                                        dataset=dataset, 
                                        DIR='./results/20210207/', 
                                        threshold=300,
                                        Synapse=False,
                                        activation='sqgl',
                                        leak=0.0,
                                        positive=False)
    hyperoptimizer_syn.loop_to_threshold()

### syn posneg

In [ ]:
classes = np.load('./results/classes.npy')
classes = classes[:,2]

for selector, dataset in enumerate(classes):
    # Initialize hyperoptimizer
    hyperoptimizer_syn = Hyperoptimizer(model_type='fcnn',
                                        selector=selector, 
                                        dataset=dataset, 
                                        DIR='./results/20210207/', 
                                        threshold=300,
                                        Synapse=True,
                                        activation='sqgl',
                                        leak=0.0,
                                        positive=False)
    hyperoptimizer_syn.loop_to_threshold()


## Best Learning Rates

In [9]:
# Get list of classes
classes = np.load('./results/classes.npy', allow_pickle=True)
class_names = classes[:,2]


activations = ['relu', 'lrelu','sigmoid','swish','sqgl']

syn_keys = ['syn', 'nosyn']
pos_keys = ['posneg']

# Initialize dictionary
sqgl_dict = {}

# If synapse included, load from correct folder by assigning DIR path
DIR = './results/20201229/'
DIR_sqgl = './results/20210207/'
    
    
# Load best hyperparameters for each dataset
act_dict = {}
for act in activations:

    dataset_dict = {}
    for dataset in class_names:
        syn_dict = {}
        if act == 'sqgl':
            for syn_key in syn_keys:
                pos_dict = {}
                for pos_key in pos_keys:
                    trials = pickle.load(open(DIR_sqgl + 'fcnn_'+act+'_'+syn_key+'_'+pos_key+'_0.0_'+dataset+'.hyperopt', "rb"))
                    lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                    pos_dict[pos_key] = {'lr' : lr}
                syn_dict[syn_key] = pos_dict
        else:
            syn_key = 'nosyn'
            
            if act == 'lrelu':
                trials = pickle.load(open(DIR + 'l_fcnn_relu_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}
            else:
                trials = pickle.load(open(DIR + 'fcnn_'+act+'_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}

        dataset_dict[dataset] = syn_dict

    act_dict[act] = dataset_dict

for act in activations:
    print(act)
    for dataset in class_names:
        
        print(dataset, act_dict[act][dataset])
        
        
with open('./results/hyperparameters/20210207_fcnn_lr.hyperopt', 'wb') as handle:
    pickle.dump(act_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

relu
mnist {'nosyn': {'lr': 0.0011}}
fmnist {'nosyn': {'lr': 0.0011}}
kmnist {'nosyn': {'lr': 0.001}}
emnist {'nosyn': {'lr': 0.0051}}
svhn {'nosyn': {'lr': 0.001}}
usps {'nosyn': {'lr': 0.0055}}
cifar10 {'nosyn': {'lr': 0.0059}}
lrelu
mnist {'nosyn': {'lr': 0.0045}}
fmnist {'nosyn': {'lr': 0.008}}
kmnist {'nosyn': {'lr': 0.0052}}
emnist {'nosyn': {'lr': 0.0095}}
svhn {'nosyn': {'lr': 0.0073}}
usps {'nosyn': {'lr': 0.0033}}
cifar10 {'nosyn': {'lr': 0.0082}}
sigmoid
mnist {'nosyn': {'lr': 0.0008}}
fmnist {'nosyn': {'lr': 0.0039}}
kmnist {'nosyn': {'lr': 0.0069}}
emnist {'nosyn': {'lr': 0.0047}}
svhn {'nosyn': {'lr': 0.0067}}
usps {'nosyn': {'lr': 0.0082}}
cifar10 {'nosyn': {'lr': 0.0059}}
swish
mnist {'nosyn': {'lr': 0.0081}}
fmnist {'nosyn': {'lr': 0.0028}}
kmnist {'nosyn': {'lr': 0.0032}}
emnist {'nosyn': {'lr': 0.0004}}
svhn {'nosyn': {'lr': 0.0001}}
usps {'nosyn': {'lr': 0.0036}}
cifar10 {'nosyn': {'lr': 0.003}}
sqgl
mnist {'syn': {'posneg': {'lr': 0.0021}}, 'nosyn': {'posneg': {'lr